In [13]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1)


'한국과 일본 사이의 거리는 대략 1000km 정도입니다. 제 이름은 AI 어시스턴트입니다. 궁금한 것이 있으시면 언제든지 물어보세요.'

In [36]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip , items))

In [43]:
template = ChatPromptTemplate.from_messages([
    ("system",
     "당신은 list 생성 기계 입니다. 입력받은 질문들은 모두 `comma seperated list`로 답해질 것입니다. 최대 {max_items}개 만큼이요. list가 아닌 것으로는 답하지 말아 주세요."),
    ("human", "{question}")
])

prompt = template.format_messages(
    max_items = 10,
    question = "What are the colors?"
)

result = chat.predict_messages(prompt)
p = CommaOutputParser()
p.parse(result.content)

['red',
 'blue',
 'green',
 'yellow',
 'orange',
 'purple',
 'pink',
 'black',
 'white',
 'brown']

In [ ]:
# chain 버전
chain = template | chat | CommaOutputParser()
chain.invoke({
    "max_items" : 5,
    "question" : "What are the colors?"
})

['red', 'blue', 'green', 'yellow', 'orange']